In [ ]:
import numpy as np 
from sklearn import datasets
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
def sigmoid(array: np.ndarray) -> np.ndarray:
    sign = np.sign(array)
    exp = np.exp(-sign * array)

    num = np.where(sign < 0, exp, 1)
    den = 1 + exp

    return num / den

In [ ]:
iris = datasets.load_iris()
samples = iris.data[:, :2]  # we only take the first two features.
gt_classes = iris.target

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.legend()

In [ ]:
logreg = LogisticRegression(solver="newton-cg")
logreg.fit(samples, (gt_classes.copy() == 0).astype(int))

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
DecisionBoundaryDisplay.from_estimator(
    logreg,
    samples,
    cmap=plt.cm.Paired,
    ax=ax,
    response_method="predict",
    plot_method="pcolormesh",
    shading="auto",
    xlabel="Sepal length",
    ylabel="Sepal width",
    eps=0.5,
)

for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.legend()

### Maximum Likelihood

In [ ]:
nsamples = samples.shape[0]
X = np.concat([samples.T, np.ones((1, nsamples))], axis=0)
W = gt_classes.copy() == 0
W = W.astype(int)
one_minus_W = 1 - W
XXT = np.einsum("ij, kj -> ikj", X, X)

In [ ]:
def objective(phi: np.ndarray) -> np.ndarray:
    phi_X = np.dot(phi, X)

    sign = np.sign(phi_X)
    exp = np.exp(-sign * phi_X)
    log1pexp = np.log1p(exp)

    obj = one_minus_W * phi_X
    obj += np.where(sign >= 0, log1pexp, log1pexp - phi_X)

    return np.sum(obj) / nsamples

def objective_prime(phi: np.ndarray) -> np.ndarray:
    sig_minus_W = sigmoid(np.dot(phi, X)) - W
    return np.einsum("j, ij -> i", sig_minus_W, X) / nsamples

def objective_second(phi: np.ndarray) -> np.ndarray:
    sig_phi_X = sigmoid(np.dot(phi, X))
    return np.einsum("k, ijk -> ij", sig_phi_X * (1 - sig_phi_X), XXT) / nsamples

In [ ]:
opt_results = minimize(
    fun=objective,
    x0=np.full((X.shape[0],), 0),
    method="Newton-CG",
    jac=objective_prime,
    hess=objective_second,
    options={"maxiter": 100},
)
print(opt_results)

phi_opt = opt_results.x

In [ ]:
xrange = np.array([4, 8])
y_lim = - (phi_opt[0] * xrange + phi_opt[2]) / phi_opt[1]

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.plot(xrange, y_lim, "r")
ax.legend()